In [1]:
import pandas as pd
from pyproj import Proj
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn import model_selection as cross_validation
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set(color_codes=True)
import collections

%matplotlib inline

In [48]:
ls data/

DOITT_SUBWAY_ENTRANCE_01_13SEPT2010.csv
Queens Light Rail BBL.csv
dtm/
finance_sales/
merged/
nyc_pluto_16v1/
open_nyc/


In [38]:
data_path = "data/merged/bronx_brooklyn_manhattan_queens_statenisland_2003_2016.csv"
df = pd.read_csv(data_path, low_memory = True)
df.head()

,zipcode_x,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,...,lottype_1.0,lottype_2.0,lottype_3.0,lottype_4.0,lottype_5.0,lottype_6.0,lottype_7.0,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2,tax_class_at_time_of_sale_3
0,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,10004.0,0.0,0.0,0.0,2725731.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [11]:
no_dups = df.drop_duplicates()
no_dups.shape

(186815, 303)

In [60]:
df.shape

(187896, 201)

In [62]:
df[['bbl','zipcode','latitude','longitude']].to_csv("bbls_and_zips.csv", index = False)

In [130]:
x = df[df['year_built'] != df['yearbuilt']]
x[x['year_built'] == 0.0].shape

(797180, 180)

In [90]:
df[['gross_sqft_pluto','sale_price','price_per_sqft']].head()

In [76]:
df = df[df['price_per_sqft'] != 0.0]
df.shape

(1583312, 180)

In [91]:
df = df[df['price_per_sqft'] >= 10]
df = df[df['price_per_sqft'] <= 5000]
df.shape

(573544, 176)

In [78]:
count = collections.Counter(df['price_per_sqft'].astype(int))
count

Counter({0: 593176,
         1: 110344,
         2: 108648,
         3: 82488,
         4: 69512,
         5: 54444,
         6: 45828,
         7: 41824,
         8: 34420,
         9: 31460,
         10: 26272,
         11: 24484,
         12: 21100,
         13: 19772,
         14: 18024,
         15: 16560,
         16: 16648,
         17: 14932,
         18: 13576,
         19: 12780,
         20: 11332,
         21: 10972,
         22: 9960,
         23: 9720,
         24: 9552,
         25: 8276,
         26: 7676,
         27: 7812,
         28: 7424,
         29: 6604,
         30: 6808,
         31: 6196,
         32: 5880,
         33: 5840,
         34: 5256,
         35: 4864,
         36: 4928,
         37: 4656,
         38: 4660,
         39: 3780,
         40: 4272,
         41: 3688,
         42: 4092,
         43: 3812,
         44: 3428,
         45: 3480,
         46: 3512,
         47: 2900,
         48: 2776,
         49: 2952,
         50: 3188,
         51: 277

In [80]:
outliers = df.loc[df['price_per_sqft'].astype(int).isin([1,2,3,4,5,6912,7655,7949,10315,12089,14568])]
outliers[['latitude','longitude','price_per_sqft','sale_price','year_built','sale_date']].to_csv("outliers.csv")

In [65]:
#output distribution of target variable to visualize in Tableau
import csv
x = df['price_per_sqft']
count = collections.Counter(x.astype(int))
with open("price_per_sqft_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Price Per Sqft', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [68]:
#output distribution of sale price to visualize in Tableau
import csv
x = df['sale_price']
count = collections.Counter(x.astype(int))
with open("sale_price_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Sale Price', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [4]:
def drop_cols(data, cols):
    return data.drop(cols, axis = 1)

In [39]:
df = drop_cols(df, ['zonemap','sale_date','sale_price','year_built'])

In [10]:
from sklearn import model_selection
def split_data(data):
    '''
    Splits data into training and test sets (0.8/0.2)
        Args: 
            data: Pandas dataframe
        Returns:
            data_train: Pandas dataframe used for training
            data_test: Pandas dataframe used for testing
    
    '''
    #Convert 'int64' into float; otherwise, sklearn throws a warning message
    columns = data.columns.values
    non_float = []
    for col in columns:
        if data[col].dtype != np.float64:
            non_float.append(col)
    for col in non_float:
        data[col] = data[col].astype(float)
    #drop NaN for crucial columns
    data= data.dropna(how = 'any', subset = ['price_per_sqft'])   
    #Split the data
    rs = model_selection.ShuffleSplit(train_size = 0.8, test_size=.2, random_state = 1, n_splits = 1)

    for train, test in rs.split(data):
        train_index = train
        test_index = test
    data_train = data.ix[train_index,:]
    data_test = data.ix[test_index,:]
    data_train.reset_index(drop=True, inplace=True)
    data_test.reset_index(drop=True, inplace=True)
    return data_train, data_test


In [40]:
data_train, data_test = split_data(df)

In [12]:
def fill_na(data_train, data_test):
    '''
    Fills NaN values with the mean of the column. Note we have already created dummy variables
    for columns with missing values.
    
    Args:
        data_train: Pandas dataframe used for training.
        data_test: Pandas dataframe used for testing.
    Returns:
        data_train: Pandas dataframe with no NaN values, ready for modeling.
        data_test: Pandas dataframe with no NaN values, ready for testing.
    
    '''
    data_train = data_train.apply(lambda x: x.fillna(x.mean()),axis=0)
    data_test = data_test.apply(lambda x: x.fillna(x.mean()),axis=0)
    return data_train, data_test

In [41]:
data_train, data_test = fill_na(data_train, data_test)

In [42]:
print(data_train.shape, data_test.shape)


(150316, 201) (37580, 201)


In [43]:
cols = list(data_train.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('price_per_sqft')) #Remove b from list
data_train = data_train[cols+['price_per_sqft']]
data_test = data_test[cols+['price_per_sqft']]
data_train.columns.values

array(['zipcode_x', 'ltdheight', 'splitzone', 'easements', 'comarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'irrlotcode',
       'bsmtcode', 'yearbuilt', 'builtcode', 'histdist', 'landmark',
       'bbl_x', 'xcoord', 'ycoord', 'latitude', 'longitude',
       'gross_sqft_pluto', 'garage', 'extension', 'countalter',
       'residential_units', 'commercial_units', 'total_units', 'bbl_y',
       'zipcode_y', 'subwaydist', 'bbl', 'zipcode',
       'botanical_gardens_dist', 'path_stations_dist', 'libraries_dist',
       'colleges_dist', 'sidewalk_cafes_dist', 'day_care_centers_dist',
       'dsny_special_waste_dist', 'link_nyc_centers_dist',
       'beacon_centers_dist', 'out_of_school_youth_centers_dist',
       'runaway_homeless_youth_centers_dist',
       'agency_service_centers_dist',
       'summer_youth_employment_centers_dist',
       'adult_continuing_ed_centers_dist',
       'young_adult_internsh

In [44]:
data_train = data_train.drop(['public_recycling_bins_dist'], axis = 1)
data_test = data_test.drop(['public_recycling_bins_dist'], axis = 1)

In [45]:
X_train = data_train.ix[:,:-1]
y_train = data_train.ix[:,-1]
X_test = data_test.ix[:,:-1]
y_test = data_test.ix[:,-1]
regr = linear_model.LinearRegression()
#regr.fit(X_train, y_train)
#mse = mean_squared_error(y_test, regr.predict(X_test))
#print('Mean_squared_error', mse)

In [20]:
from sklearn.ensemble import RandomForestRegressor

RF_reg_final = RandomForestRegressor(n_estimators=100, n_jobs = -1)
RF_reg_final.fit(X_train, y_train)
print(mean_squared_error(y_test, RF_reg_final.predict(X_test)))

998658.494381


In [32]:
bbls = []
file = open("data/Queens Light Rail BBL.csv", 'rb')
for line in file:
    bbls.append(line)
bbls = [float(i) for i in bbls]
bbls

[4000340047.0,
 4038530098.0,
 4027260002.0,
 4026930121.0,
 4033840065.0,
 4038440018.0,
 4036170028.0,
 4032630033.0,
 4027510038.0,
 4039010063.0,
 4031790047.0,
 4032600003.0,
 4036110037.0,
 4092080136.0,
 4036890066.0,
 4027110014.0,
 4036620055.0,
 4027280068.0,
 4035320042.0,
 4000480046.0,
 4000570014.0,
 4091820042.0,
 4093010100.0,
 4092310044.0,
 4093210003.0,
 4096990048.0,
 4037740014.0,
 4000780136.0,
 4027480062.0,
 4027660068.0,
 4026220125.0,
 4035130039.0,
 4002340023.0,
 4036010037.0,
 4093530019.0,
 4032200065.0,
 4038950062.0,
 4035190042.0,
 4026880067.0,
 4093180070.0,
 4092730075.0,
 4094440003.0,
 4036590015.0,
 4096760007.0,
 4036950030.0,
 4091840041.0,
 4036490041.0,
 4096310020.0,
 4026410027.0,
 4036180013.0,
 4093280075.0,
 4099700029.0,
 4027260043.0,
 4091940039.0,
 4026750134.0,
 4025350033.0,
 4038260052.0,
 4026920054.0,
 4000750007.0,
 4093460003.0,
 4037880005.0,
 4026640030.0,
 4026800041.0,
 4035130038.0,
 4033820032.0,
 4096880027.0,
 402758006

In [67]:
finance = pd.read_csv("finance_queens.csv")
finance.shape

(312142, 9)

In [64]:
queens = pd.read_csv("data/nyc_pluto_16v1/QN.csv")
queens.head()

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (18,19,21,22,65,66,78,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version
0,QN,6,1,402,1.0,NaN,30.0,26.0,11101.0,L115,...,NaN,8d,Y,401 011,40101.0,NaN,4.000060e+09,9/20/2013,1,16v1
1,QN,6,3,402,1.0,1015.0,30.0,26.0,11101.0,L115,...,209421.0,8d,NaN,401 011,40101.0,NaN,0.000000e+00,NaN,1,16v1
2,QN,6,8,402,1.0,1011.0,30.0,26.0,11101.0,L115,...,209867.0,8d,NaN,401 011,40101.0,NaN,4.000060e+09,8/7/2013,1,16v1
3,QN,6,20,402,1.0,NaN,30.0,26.0,11101.0,L115,...,NaN,8d,NaN,401 011,40101.0,NaN,4.000060e+09,9/20/2013,1,16v1
4,QN,6,30,402,1.0,NaN,30.0,26.0,11101.0,L115,...,NaN,8d,NaN,401 011,40101.0,NaN,4.000060e+09,9/20/2013,1,16v1


In [74]:
buildings_2 = pd.read_csv("pluto_finance_test.csv")
buildings_2.shape

(213239, 50)

In [86]:
buildings_3 = pd.read_csv("data/merged/queens_2003_2016.csv")
buildings_3['bbl_y'].isnull().sum()

213239

In [89]:
buildings_3['bbl']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
          ..
213209   NaN
213210   NaN
213211   NaN
213212   NaN
213213   NaN
213214   NaN
213215   NaN
213216   NaN
213217   NaN
213218   NaN
213219   NaN
213220   NaN
213221   NaN
213222   NaN
213223   NaN
213224   NaN
213225   NaN
213226   NaN
213227   NaN
213228   NaN
213229   NaN
213230   NaN
213231   NaN
213232   NaN
213233   NaN
213234   NaN
213235   NaN
213236   NaN
213237   NaN
213238   NaN
Name: bbl, dtype: float64

In [69]:
buildings = pd.merge(queens, finance, how='right',
        left_on='BBL', right_on = 'bbl')
buildings.shape

(312142, 95)

In [76]:
buildings_2 = buildings_2.drop_duplicates()
buildings_2.shape

(213112, 50)

In [54]:
affected_properties = finance.loc[finance['bbl'].isin(bbls)]
affected_properties.shape

(0, 9)

In [63]:
list(X_test['bbl'])

[1007937505.0,
 1012940033.0,
 3032580030.0,
 1012450047.0,
 1005530039.0,
 3045650090.0,
 1020590007.0,
 1016390029.0,
 3059780050.0,
 1021710120.0,
 3062690010.0,
 1010087503.0,
 3016830016.0,
 3061290104.0,
 3015960069.0,
 1007677504.0,
 3060250058.0,
 1012460115.0,
 1011717502.0,
 1004980018.0,
 1008760001.0,
 3014180065.0,
 1020140013.0,
 1010550055.0,
 1010087503.0,
 1014990020.0,
 1014667501.0,
 1011397503.0,
 1007200021.0,
 1015377501.0,
 3039290051.0,
 3061860031.0,
 1021160052.0,
 3082910024.0,
 3080300033.0,
 3059370050.0,
 1018467501.0,
 1004510025.0,
 1011157501.0,
 3018390048.0,
 1012100040.0,
 1002197504.0,
 1000167504.0,
 1010277504.0,
 1015377501.0,
 1007677504.0,
 3019420027.0,
 1000167504.0,
 1015367501.0,
 1015367501.0,
 1011190047.0,
 3061550062.0,
 1006150025.0,
 1007320016.0,
 1011850045.0,
 1014647501.0,
 1014520010.0,
 3084350070.0,
 1011717501.0,
 1020240013.0,
 1011377501.0,
 1009117501.0,
 1008300080.0,
 1011407501.0,
 1001750004.0,
 1006210013.0,
 306060000

In [145]:
feature_importance =  RF_reg_final.feature_importances_
indices = np.argsort(feature_importance)[::-1][:27]

feature_dct = {}
# Print the feature ranking
print("Feature ranking:")

for f in range(20):
    feature_dct[X_test.columns.values[indices][f]] = feature_importance[indices[f]]
feature_dct

Feature ranking:


{'bldgfront': 0.018375322795829107,
 'comarea': 0.010972858127390312,
 'commercial_units': 0.032859378378804184,
 'condono': 0.026726479840308241,
 'council_4.0': 0.072414961700190308,
 'gross_sqft_pluto': 0.28185238649272115,
 'latitude': 0.01574201393763933,
 'longitude': 0.0093627575879397934,
 'lotdepth': 0.0079779554635258793,
 'lotfront': 0.016403509960369191,
 'numfloors': 0.018516345028803366,
 'resarea': 0.016554750466656002,
 'residential_units': 0.01445442649834509,
 'tax_class_at_time_of_sale_2': 0.0097889932832402552,
 'unitstotal': 0.25183258094173272,
 'unitstotal_mv': 0.020185006969104351,
 'xcoord': 0.0095951270740613579,
 'ycoord': 0.017265038087155543,
 'year_built': 0.052904916514372152,
 'yearbuilt': 0.029749012331112573}

In [115]:
from collections import OrderedDict
from operator import itemgetter

feature_dct = OrderedDict(sorted(feature_dct.items(), key=itemgetter(1), reverse = True))

In [116]:
feature_dct.keys()

odict_keys(['gross_sqft_pluto', 'unitstotal', 'council_4.0', 'year_built', 'yearbuilt', 'commercial_units', 'condono', 'ycoord', 'bldgfront', 'latitude', 'lotfront', 'numfloors', 'resarea', 'unitstotal_mv', 'residential_units', 'comarea', 'unitsres_mv', 'xcoord', 'tax_class_at_time_of_sale_2', 'longitude'])

In [121]:
data_train[['price_per_sqft'] + list(feature_dct.keys())].corr()

,price_per_sqft,gross_sqft_pluto,unitstotal,council_4.0,year_built,yearbuilt,commercial_units,condono,ycoord,bldgfront,...,lotfront,numfloors,resarea,unitstotal_mv,residential_units,comarea,unitsres_mv,xcoord,tax_class_at_time_of_sale_2,longitude
price_per_sqft,1.000000,-0.155891,-3.069134e-01,-0.041846,0.008975,-0.101872,0.119171,0.014776,-0.102872,-0.007528,...,-0.102020,-0.161124,-0.212818,-2.088541e-01,0.026022,-0.017072,-0.206464,-0.089052,-0.161219,-0.089078
gross_sqft_pluto,-0.155891,1.000000,1.288403e-01,0.064277,0.027628,0.205999,-0.016409,0.171342,0.209837,-0.056400,...,0.011188,0.625938,0.861798,5.418245e-01,0.054894,0.757434,0.534673,0.083097,-0.008056,0.083169
unitstotal,-0.306913,0.128840,1.000000e+00,0.051115,0.074670,0.135439,-0.134089,0.006081,0.173348,0.035438,...,0.138483,0.175816,0.179090,-4.870732e-17,0.007974,0.009989,0.015934,0.123647,0.100898,0.123703
council_4.0,-0.041846,0.064277,5.111517e-02,1.000000,0.016434,0.068393,-0.007452,-0.037177,0.144517,-0.052784,...,0.073537,0.327979,0.061006,1.964187e-01,-0.000073,0.041462,0.203593,0.303901,-0.008175,0.303862
year_built,0.008975,0.027628,7.467030e-02,0.016434,1.000000,0.126385,-0.022570,0.148423,-0.009951,0.135440,...,-0.003781,0.067843,0.065526,7.412231e-02,0.002087,-0.032768,0.061816,-0.001348,0.018588,-0.001355
yearbuilt,-0.101872,0.205999,1.354395e-01,0.068393,0.126385,1.000000,-0.043343,0.198477,0.089574,0.051260,...,0.060470,0.321878,0.241294,2.198613e-01,0.021065,0.073956,0.209554,0.061418,0.042581,0.061394
commercial_units,0.119171,-0.016409,-1.340893e-01,-0.007452,-0.022570,-0.043343,1.000000,-0.002974,-0.080044,-0.021882,...,-0.036792,-0.041131,-0.102607,-6.517582e-02,-0.033339,0.101441,-0.097546,-0.064849,-0.626440,-0.064860
condono,0.014776,0.171342,6.080814e-03,-0.037177,0.148423,0.198477,-0.002974,1.000000,-0.111436,0.394462,...,0.044822,0.174171,0.227655,3.507324e-02,0.039002,0.026818,0.011836,-0.133908,-0.005096,-0.133951
ycoord,-0.102872,0.209837,1.733480e-01,0.144517,-0.009951,0.089574,-0.080044,-0.111436,1.000000,-0.092808,...,-0.043004,0.241307,0.242505,2.215443e-01,0.004390,0.079561,0.201184,0.751065,0.078466,0.751244
bldgfront,-0.007528,-0.056400,3.543771e-02,-0.052784,0.135440,0.051260,-0.021882,0.394462,-0.092808,1.000000,...,0.342909,-0.045378,-0.051765,-1.437557e-01,0.011334,-0.038653,-0.137885,-0.031337,0.008300,-0.031356


In [ ]:
neg_features = feature_dct.copy()
pos_features = feature_dct.copy()

#Use correlation matrix to determine which features are negatively correlated with our target variable
negs = ['gross_sqft_pluto','unitstotal']
for key in neg_features.keys():
    if key in negs:
        neg_features[key] = -neg_features[key]
    else:
        neg_features[key] = 0
for key in pos_features.keys():
    if key in negs:
        pos_features[key] = 0

In [103]:
data_test['predicted'] = RF_reg_final.predict(X_test)
data_test['percent_difference'] = 100*(np.abs(data_test['predicted'] - data_test['price_per_sqft']).astype(float) / data_test['price_per_sqft'])
data_test[['yearbuilt','price_per_sqft','predicted','percent_difference']]

,yearbuilt,price_per_sqft,predicted,percent_difference
0,1947.507154,64.230643,60.960564,5.091151
1,1925.000000,593.525180,364.947894,38.511809
2,1947.507154,64.230643,60.960564,5.091151
3,1947.507154,64.230643,60.960564,5.091151
4,1947.507154,64.230643,60.960564,5.091151
5,1947.507154,64.230643,60.960564,5.091151
6,1947.507154,64.230643,60.960564,5.091151
7,1962.000000,59.000000,82.196851,39.316696
8,1947.507154,64.230643,60.960564,5.091151
9,1947.507154,64.230643,60.960564,5.091151


In [102]:
acc = 100 * (data_test[data_test['percent_difference'] < 10.0].shape[0]/ data_test.shape[0])
acc

73.76945154360696

In [28]:
results_df = pd.read_csv("preliminary_results.csv")
results_df

,Model Setup,Unnamed: 1,MSE,Accuracy
0,Linear Regression,LR - All records,4976.12,5.43%
1,NaN,LR - Price Per Sqft > 0,3867.97,8.56%
2,NaN,LR - Price Per Sqft > 5,3076.29,27.64%
3,Random Forest,RF - All records,1359.81,13.82%
4,NaN,RF Price Per Sqft > 0,3835.71,64.46%
5,NaN,RF - Price Per Sqft > 5,730.32,69.66%
